<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png"/>

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>    

<a href="http://colab.research.google.com/github/dipanjanS/nlp_workshop_odsc19/blob/master/Module05%20-%20NLP%20Applications/Project07B%20-%20Text%20Classification%20Deep%20Learning%20CNN%20Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cómo crear una matriz de word embeddings



El tema anterior estuvo dedicado a implementar un modelo CNN para abordar la tarea de clasificación. 
En dicho modelo, los tokens eran reemplazados por vectores creados de forma random (internamente lo hace la capa Embedding). 

En este notebook, aprenderemos a crear una matriz de word embeddings. La matriz tendrá tantas filas como número de palabras distintas en nuestra colección de textos. Cada fila será el word embedding para una determinada palabra recuperado de un modelo pre-entrenado de word embedding. 

Vamos a utilizar el dataset EXIST. Recuerda que debemos el dataset debe solicitarse a los organizadores de la competición: jcalbornoz@lsi.uned.es, frodriguez.sanchez@invi.uned.es or lplaza@lsi.uned.es

The dataset should only be used for scientific or research purposes. Any other use is explicitly prohibited.
The datasets must not be redistributed or shared in part or full to any third party. Redirect interested parties to this website.
All data released for these tasks are under the CC BY-SA 4.0 License (https://creativecommons.org/licenses/by-sa/4.0/legalcode).
Data must not be used for providing surveillance, analyses or research that isolates a group of individuals or any single individual for any unlawful or discriminatory purpose.


Vamos a cargar únicamente los textos en inglés del conjunto de entrenamiento:

In [17]:
from google.colab import drive
# monta disco de google drive
drive.mount('/content/drive')

# Indica la ruta de la carpeta de Google Drive
# donde está tu dataset
PATH = "/content/drive/My Drive/Colab Notebooks/data/exist2022/"

import pandas as pd
df_train = pd.read_csv(PATH+"EXIST2021_training.tsv",  sep='\t') 

df_train = df_train[df_train['language']=='en']
print('tamaño training:', df_train.shape)
df_train.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
tamaño training: (3436, 7)


,test_case,id,source,language,text,task1,task2
0,EXIST2021,1,twitter,en,"She calls herself ""anti-feminazi"" how about sh...",sexist,ideological-inequality
1,EXIST2021,2,twitter,en,"Now, back to these women, the brave and the be...",non-sexist,non-sexist
2,EXIST2021,3,twitter,en,"@CurvyBandida @Xalynne_B Wow, your skirt is ve...",sexist,objectification
3,EXIST2021,4,twitter,en,@AurelieGuiboud Incredible! Beautiful!But I l...,non-sexist,non-sexist
4,EXIST2021,5,twitter,en,i find it extremely hard to believe that kelly...,non-sexist,non-sexist


In [18]:
length_train = df_train['text'].apply(lambda text: len(text))
length_train.describe()

count    3436.000000
mean      176.182771
std       102.405307
min        13.000000
25%        89.750000
50%       171.000000
75%       262.000000
max       962.000000
Name: text, dtype: float64

Vamos a entrenar el tokenizador de keras con los textos del conjunto de entrenamiento:

In [19]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences 

tokenizer = Tokenizer(oov_token = True)
X_train = list(df_train['text'])

tokenizer.fit_on_texts(X_train)
tokenizer.word_index['<PAD>'] = 0

NUM_WORDS=len(tokenizer.word_index)
print("Tamaño del vocabulario ={}".format(NUM_WORDS))

MAX_LEN = 300
print("Longitud secuencia máxima ={}".format(MAX_LEN))


Tamaño del vocabulario =16645
Longitud secuencia máxima =300


Únicamente hemos cargado los textos en inglés, porque vamos a cargar un modelo pre-entrenado de word embeddings para el inglés. 

Vamos a crear ahora la matriz de word embeddings. Para ello cargamos un modelo pre-entrenado con word embeddings para inglés. Es un modelo pequeño para que se cargue rapidamente. Puedes probar con otros modelos de word embeddings (https://github.com/RaRe-Technologies/gensim-data).

Una vez cargado el modelo, inicializamos la matriz con 0. La matriz tiene tantas filas como palabras en el vocabulario creado por el tokenizador a partir de la colección de textos. 
El número de columnas es igual al tamaño de los vectores. 

Vamos a recorrer cada palabra del vocabulario, que tiene asociado un entero, i, y para cada palabra vamos a recuperar su vector del modelo pre-entrenado, para asociarlo a la fila i. Si la palabra no existe el modelo, se lanzaría una excepción que nosotros capturamos y no hacemos anda. El vector de esa palabra sería todo 0's. 


In [ ]:
import gensim.downloader as api
import numpy as np

model_we = api.load("glove-wiki-gigaword-50")
#model_we = api.load("word2vec-google-news-300")     #EMBED_SIZE = 300

EMBED_SIZE = 50

# creamos una matriz para los textos del conjunto de entrenamiento
embedding_matrix = np.zeros((NUM_WORDS, EMBED_SIZE))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = model_we[word]
        # word embedding para la palabra con índice i
        embedding_matrix[i] = embedding_vector
    except:
        #if word does not exist, we do not udpate the matrix
        pass

print('matriz creada')



Mostramos las 10 primeras palabras del vocabulario y sus vectores

In [22]:
print(embedding_matrix.shape)
for word, i in tokenizer.word_index.items():
    print(word, "word embedding:", embedding_matrix[i])
    if i==10:
        break


(16645, 50)
True word embedding: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
the word embedding: [ 4.18000013e-01  2.49679998e-01 -4.12420005e-01  1.21699996e-01
  3.45270008e-01 -4.44569997e-02 -4.96879995e-01 -1.78619996e-01
 -6.60229998e-04 -6.56599998e-01  2.78430015e-01 -1.47670001e-01
 -5.56770027e-01  1.46579996e-01 -9.50950012e-03  1.16579998e-02
  1.02040000e-01 -1.27920002e-01 -8.44299972e-01 -1.21809997e-01
 -1.68009996e-02 -3.32789987e-01 -1.55200005e-01 -2.31309995e-01
 -1.91809997e-01 -1.88230002e+00 -7.67459989e-01  9.90509987e-02
 -4.21249986e-01 -1.95260003e-01  4.00710011e+00 -1.85939997e-01
 -5.22870004e-01 -3.16810012e-01  5.92130003e-04  7.44489999e-03
  1.77780002e-01 -1.58969998e-01  1.20409997e-02 -5.42230010e-02
 -2.98709989e-01 -1.57490000e-01 -3.47579986e-01 -4.56370004e-02
 -4.42510009e-01  1.87849998e-01  2.78489990e-03 -1.84110001e-01
 -1.15139998e

¿Cómo se utiliza esta matriz?. Se utiliza en la primera capa del modelo, es decir, en la capa Embedding. La matriz se le pasa al parámetro **weights**.
                     

```
# Capa embeddings
model.add(Embedding(NUM_WORDS, EMBED_SIZE, 
                        input_length=MAX_LEN,
                        weights=[embedding_matrix]))
```

